In [ ]:
#Tensorflow__ImageGenerator_DNN_Prf

In [ ]:
# Download data
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

In [ ]:
# Unzip data
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir
path = '/Users/farnaz/Documents/TF/happy-or-sad.zip'
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall('/Users/farnaz/Documents/TF/happy-or-sad')
zip_ref.close()

In [ ]:
#plot data

train_happy_dir = os.path.join('/Users/farnaz/Documents/TF/happy-or-sad/happy')
train_sad_dir = os.path.join('/Users/farnaz/Documents/TF/happy-or-sad/sad')
train_happy_names = os.listdir(train_happy_dir)
train_sad_names = os.listdir(train_sad_dir)

print('total training horse images:', len(os.listdir(train_happy_dir)))
print('total training human images:', len(os.listdir(train_sad_dir)))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 4
ncols = 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_happy_pix = [os.path.join(train_happy_dir, fname) 
                for fname in train_happy_names[pic_index-8:pic_index]]
next_sad_pix = [os.path.join(train_sad_dir, fname) 
                for fname in train_sad_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_happy_pix+next_sad_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [2]:
# Building the Model 
import tensorflow as tf
print(tf.__version__)

def train_happy_sad_model():
    
    # Step 5 define class 
    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')> 0.999):
            self.model.stop_training = True 
        
    # Step 4 define callbacks    
    callbacks = myCallback()
    # Step 2 define the model
    model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.summary()

    from tensorflow.keras.optimizers import RMSprop
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
    
    # Step 1 load and normalize data 
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1/255)
    train_generator = train_datagen.flow_from_directory('/Users/farnaz/Documents/TF/happy-or-sad/', 
                                                    target_size=(150, 150),batch_size=10,class_mode='binary')
    # Step 3 feed data to the model; training
    history = model.fit(train_generator,steps_per_epoch=8,epochs=15,verbose=1,callbacks=[callbacks])

    return history.history['accuracy'][-1]

train_happy_sad_model()

2.0.0-alpha0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496) 

1.0